In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/jianingyu/Desktop/scent-to-scene/scent-to-scene.csv")

cue_cols = [
    "cue_disinfectant","cue_rubber","cue_iodine",
    "cue_butter","cue_vanilla","cue_strawberry",
    "cue_grass","cue_wood","cue_lily"
]
abs_cols = ["img_clean","img_sweet","img_natural","img_chemical"]

X_cue = df[cue_cols].to_numpy(dtype=float)
I_abs_raw = df[abs_cols].to_numpy(dtype=float)
I_abs = (I_abs_raw - I_abs_raw.mean(axis=0)) / (I_abs_raw.std(axis=0) + 1e-8)

# ---- encode scene label y: 0/1/2 ----
scene_order = ["hospital","bakery","garden"]
y = df["scene_choice"].astype(pd.CategoricalDtype(categories=scene_order)).cat.codes.to_numpy()

# optional: concrete imagery Top-1 as one-hot
obj_order = ["disinfectant","rubber","iodine","butter","vanilla","strawberry","grass","wood","lily"]
obj_codes = df["img_obj_top1"].astype(pd.CategoricalDtype(categories=obj_order)).cat.codes.to_numpy()
X_obj_top1 = np.eye(len(obj_order))[obj_codes]  # (N, 9)

N = len(df)
K = len(scene_order)  # 3
D_cue = X_cue.shape[1]  # 9
D_abs = I_abs.shape[1]  # 4
